In [27]:
import os
import wandb

import pandas as pd
import tensorflow as tf

from huggingface_hub import HfApi, HfFolder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax
from tensorflow.keras.callbacks import Callback

In [2]:
path_dataset = {
    "train": "./dataset/splited_dataset/train",
    "test": "./dataset/splited_dataset/test",
}
hyperparameters = {
    "augmentation": False,
    "image_size": (128, 128),
    "batch_size": 32,
    "epochs": 10,
    "learning_rate": None,
    "optimizer": "adam",
    "loss": "sparse_categorical_crossentropy",
    "metrics": ["accuracy"],
}
optimizer_map = {
    "adam": Adam(),
    "sgd": SGD(),
    "rmsprop": RMSprop(),
    "adamax": Adamax(),
}
labels = os.listdir(path_dataset["train"])

In [13]:
def get_imgs_path(path:str, labels:list=labels):
    imgs_path = {"img_path": [], "label": []}
    for label in labels:
        imgs_path["img_path"] += [os.path.join(path, label, file) for file in os.listdir(os.path.join(path, label))]
        imgs_path["label"] += [label for _ in range(len(os.listdir(os.path.join(path, label))))]
    imgs_path = pd.DataFrame(imgs_path)
    imgs_path["label"] = imgs_path["label"].map({label: i for i, label in enumerate(labels)})
    return pd.DataFrame(imgs_path)

In [4]:
def load_img(path:str, label:str, image_size:tuple=hyperparameters["image_size"]):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, image_size)
    img = tf.cast(img, tf.float32) / 255.
    return img, label

In [15]:
path_imgs_train = get_imgs_path(path_dataset["train"])
path_imgs_test = get_imgs_path(path_dataset["test"])

In [16]:
trainset = tf.data.Dataset.from_tensor_slices((path_imgs_train["img_path"], path_imgs_train["label"]))
trainset = trainset.map(load_img)
trainset = trainset.shuffle(len(path_imgs_train)).batch(32)

testset = tf.data.Dataset.from_tensor_slices((path_imgs_test["img_path"], path_imgs_test["label"]))
testset = testset.map(load_img)
testset = testset.shuffle(len(path_imgs_test)).batch(32)

In [17]:
for img, label in testset.take(1):
    print(img.shape, label.shape)
    print(label[0])

(32, 128, 128, 3) (32,)
tf.Tensor(4, shape=(), dtype=int64)


In [18]:
wandb.init(project="trash-classification", config=hyperparameters)
config = wandb.config

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bugi-sulistiyo (bugi-sulistiyo-universitas-mulawarman). Use `wandb login --relogin` to force relogin


In [19]:
model = Sequential([
    Conv2D(16, (3, 3), activation="relu", input_shape=(config.image_size[0], config.image_size[1], 3)),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(len(path_imgs_train.label.unique()), activation="softmax")
])

In [20]:
model.compile(
    optimizer=optimizer_map[config.optimizer],
    loss=config.loss,
    metrics=config.metrics
)

In [21]:
class CustomWandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log(logs)

In [22]:
model.fit(
    trainset,
    validation_data=testset,
    epochs=config.epochs,
    callbacks=[CustomWandbCallback()]
)

Epoch 1/10


64/64 [==============================] - 163s 1s/step - loss: 3.9693 - accuracy: 0.4602 - val_loss: 2.0383 - val_accuracy: 0.2411
Epoch 2/10
64/64 [==============================] - 151s 944ms/step - loss: 1.0848 - accuracy: 0.6714 - val_loss: 1.6427 - val_accuracy: 0.2787
Epoch 3/10
64/64 [==============================] - 121s 896ms/step - loss: 0.6055 - accuracy: 0.8001 - val_loss: 1.4798 - val_accuracy: 0.3696
Epoch 4/10
64/64 [==============================] - 134s 1s/step - loss: 0.2844 - accuracy: 0.9040 - val_loss: 1.4495 - val_accuracy: 0.3557
Epoch 5/10
64/64 [==============================] - 101s 693ms/step - loss: 0.1752 - accuracy: 0.9441 - val_loss: 1.1859 - val_accuracy: 0.5632
Epoch 6/10
64/64 [==============================] - 95s 772ms/step - loss: 0.1264 - accuracy: 0.9659 - val_loss: 1.3197 - val_accuracy: 0.4625
Epoch 7/10
64/64 [==============================] - 84s 632ms/step - loss: 0.0742 - accuracy: 0.9837 - val_loss: 1.3366 - val_accuracy: 0.557

In [23]:
model.save("model/vanila_cnn.h5")
wandb.log_artifact("model/vanila_cnn.h5", type="model")

c:\Users\Bugi\miniconda3\envs\modeling\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<Artifact run-3us1ssr1-vanila_cnn.h5>

In [24]:
wandb.finish()

accuracy,▁▄▅▇▇█████
loss,█▃▂▁▁▁▁▁▁▁
val_accuracy,▁▂▃▃▇▅▇███
val_loss,█▅▃▃▁▂▂▁▃▄
accuracy,0.99604
loss,0.01647
val_accuracy,0.61858
val_loss,1.58671


In [41]:
api = HfApi()

In [47]:
repo = api.create_repo("trash-classification", repo_type="model", exist_ok=True)

In [46]:
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj="model/vanila_cnn.h5",
    path_in_repo="model/vanila_cnn.h5",
    repo_type=repo.repo_type,
    commit_message="Add model vanila_cnn.h5 (initial commit)"
)

vanila_cnn.h5: 100%|██████████| 97.6M/97.6M [01:00<00:00, 1.61MB/s]


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/trash-classification/commit/fec199e558ec0950ce934132bdd90aeb34c24a59', commit_message='Add model vanila_cnn.h5 (initial commit)', commit_description='', oid='fec199e558ec0950ce934132bdd90aeb34c24a59', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/trash-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/trash-classification'), pr_revision=None, pr_num=None)